# Bedrock Python Demo

The following is a demo of the Bedrock python package, used to ease the configuration of a Bedrock deployment using Python.

In [1]:
import sys
sys.path.append('/projects/mochi/py-bedrock') # use the local source of py-mochi-bedrock
sys.path.append('/projects/mochi/py-bedrock/build/lib.linux-x86_64-3.7')

## Getting started

The root of a Bedrock configuration is a `ProcSpec`, defined in the `bedrock.spec` package. It defines everything we want to run on a process.

In [2]:
import bedrock.spec

We can create a `ProcSpec` using a single `margo` key specifying the protocol we want to use.

In [3]:
my_process = bedrock.spec.ProcSpec(margo='na+sm')

All the specification objects have a `to_json()` method to generate their JSON configuration. Let's look at our `proc`'s configuration:

In [4]:
print(my_process.to_json(indent=4))

{
    "margo": {
        "progress_timeout_ub_msec": 100,
        "enable_profiling": false,
        "enable_diagnostics": false,
        "handle_cache_size": 32,
        "profile_sparkline_timeslice_msec": 1000,
        "argobots": {
            "abt_mem_max_num_stacks": 8,
            "abt_thread_stacksize": 2097152,
            "version": "unknown",
            "pools": [
                {
                    "name": "__primary__",
                    "kind": "fifo_wait",
                    "access": "mpmc"
                }
            ],
            "xstreams": [
                {
                    "name": "__primary__",
                    "cpubind": -1,
                    "affinity": [],
                    "scheduler": {
                        "type": "basic_wait",
                        "pools": [
                            "__primary__"
                        ]
                    }
                }
            ]
        },
        "mercury": {
            "address":

Py-bedrock gives us a very simple way of accessing sub-configurations. Objects that have a name can be accessed using their name or their index in the list that contains them. For example, accessing Argobots' primary pool:

In [5]:
print(my_process.margo.argobots.pools['__primary__'])
print(my_process.margo.argobots.pools[0])

PoolSpec(name='__primary__', kind='fifo_wait', access='mpmc')
PoolSpec(name='__primary__', kind='fifo_wait', access='mpmc')


Let's customize our process a little more. What about defining a pool for RPCs to run on, along with a few execution streams using that pool?

In [6]:
# Create the new pool, it will be added automatically to the process
my_rpc_pool = my_process.margo.argobots.pools.add(name='my_rpc_pool', kind='fifo', access='mpmc')
# Create two execution streams using that pool
for i in range(0,4):
    sched = bedrock.spec.SchedulerSpec(type='basic', pools=[my_rpc_pool])
    my_process.margo.argobots.xstreams.add(name=f'my_xstream_{i}', scheduler=sched)
# Now let's set the pools we want for handling RPCs
my_process.margo.rpc_pool = my_rpc_pool

Let's now print the resulting configuration:

In [7]:
print(my_process.to_json(indent=4))

{
    "margo": {
        "progress_timeout_ub_msec": 100,
        "enable_profiling": false,
        "enable_diagnostics": false,
        "handle_cache_size": 32,
        "profile_sparkline_timeslice_msec": 1000,
        "argobots": {
            "abt_mem_max_num_stacks": 8,
            "abt_thread_stacksize": 2097152,
            "version": "unknown",
            "pools": [
                {
                    "name": "__primary__",
                    "kind": "fifo_wait",
                    "access": "mpmc"
                },
                {
                    "name": "my_rpc_pool",
                    "kind": "fifo",
                    "access": "mpmc"
                }
            ],
            "xstreams": [
                {
                    "name": "__primary__",
                    "cpubind": -1,
                    "affinity": [],
                    "scheduler": {
                        "type": "basic_wait",
                        "pools": [
                       

## Adding ABT-IO instances

Let's add an ABT-IO instance.

In [8]:
my_process.abt_io.add(name='my_abtio', pool=my_rpc_pool)

AbtIOSpec(name='my_abtio', pool=PoolSpec(name='my_rpc_pool', kind='fifo', access='mpmc'))

## Adding SSG groups
Let's add an SSG group.

In [9]:
swim_config = bedrock.spec.SwimSpec(
    period_length_ms=1000,
    suspect_timeout_periods=3,
    subgroup_member_count=0,
    disabled=False)

my_process.ssg.add(name='my_group',
                   swim=swim_config,
                   bootstrap='init',
                   group_file='my_group.ssg',
                   pool=my_rpc_pool)

SSGSpec(name='my_group', pool=PoolSpec(name='my_rpc_pool', kind='fifo', access='mpmc'), credential=-1, bootstrap='init', group_file='my_group.ssg', swim=SwimSpec(period_length_ms=1000, suspect_timeout_periods=3, subgroup_member_count=0, disabled=False))

In [10]:
print(my_process.to_json(indent=4))

{
    "margo": {
        "progress_timeout_ub_msec": 100,
        "enable_profiling": false,
        "enable_diagnostics": false,
        "handle_cache_size": 32,
        "profile_sparkline_timeslice_msec": 1000,
        "argobots": {
            "abt_mem_max_num_stacks": 8,
            "abt_thread_stacksize": 2097152,
            "version": "unknown",
            "pools": [
                {
                    "name": "__primary__",
                    "kind": "fifo_wait",
                    "access": "mpmc"
                },
                {
                    "name": "my_rpc_pool",
                    "kind": "fifo",
                    "access": "mpmc"
                }
            ],
            "xstreams": [
                {
                    "name": "__primary__",
                    "cpubind": -1,
                    "affinity": [],
                    "scheduler": {
                        "type": "basic_wait",
                        "pools": [
                       

## Adding providers

First we need to add module libraries for our providers.

In [11]:
my_process.libraries['module_a'] = '/projects/mochi/bedrock/build/examples/libexample-module-a.so'
my_process.libraries['module_b'] = '/projects/mochi/bedrock/build/examples/libexample-module-b.so'

Now let's add providers. Py-bedrock has no idea what dependencies are required by providers of each module, to you need to make sure the dependencies and configurations are correct on your own.

In [12]:
my_process.providers.add(
    name='ProviderA',
    type='module_a',
    provider_id=42,
    pool=my_rpc_pool,
    config={},
    dependencies={})

my_process.providers.add(
    name='ProviderB',
    type='module_b',
    provider_id=33,
    pool=my_rpc_pool,
    config={},
    dependencies={
        "ssg_group" : 'my_group',
        "a_provider" : "module_a:42",
        "a_local" : [ "ProviderA@ssg://my_group/0" ],
        "a_client" : "module_a:client"
    })

ProviderSpec(name='ProviderB', type='module_b', pool=PoolSpec(name='my_rpc_pool', kind='fifo', access='mpmc'), provider_id=33, config={}, dependencies={'ssg_group': 'my_group', 'a_provider': 'module_a:42', 'a_local': ['ProviderA@ssg://my_group/0'], 'a_client': 'module_a:client'})

In [13]:
print(my_process.to_json(indent=4))

{
    "margo": {
        "progress_timeout_ub_msec": 100,
        "enable_profiling": false,
        "enable_diagnostics": false,
        "handle_cache_size": 32,
        "profile_sparkline_timeslice_msec": 1000,
        "argobots": {
            "abt_mem_max_num_stacks": 8,
            "abt_thread_stacksize": 2097152,
            "version": "unknown",
            "pools": [
                {
                    "name": "__primary__",
                    "kind": "fifo_wait",
                    "access": "mpmc"
                },
                {
                    "name": "my_rpc_pool",
                    "kind": "fifo",
                    "access": "mpmc"
                }
            ],
            "xstreams": [
                {
                    "name": "__primary__",
                    "cpubind": -1,
                    "affinity": [],
                    "scheduler": {
                        "type": "basic_wait",
                        "pools": [
                       

## Deploying the service
We may now deploy the service using a platform-specific `deploy` function.

In [14]:
import bedrock.local
bedrock.local.deploy(my_process, output=True, daemon=True)

[2020-12-01 18:35:22.784] [warning] Overriding field "mercury.version" ("unknown") with value "2.0.0"
[2020-12-01 18:35:22.793] [warning] Overriding field "argobots.version" ("unknown") with value "1.0"
[2020-12-01 18:35:22.796] [warning] ABT_xstream_get_cpubind failed to get cpubind (ret = 51)
[2020-12-01 18:35:22.807] [warning] ABT_xstream_get_affinity failed to get affinity (ret = 51)
[2020-12-01 18:35:22.807] [warning] ABT_xstream_get_cpubind failed to get cpubind (ret = 51)
[2020-12-01 18:35:22.807] [warning] ABT_xstream_get_affinity failed to get affinity (ret = 51)
[2020-12-01 18:35:22.807] [warning] ABT_xstream_get_cpubind failed to get cpubind (ret = 51)
[2020-12-01 18:35:22.807] [warning] ABT_xstream_get_affinity failed to get affinity (ret = 51)
[2020-12-01 18:35:22.808] [warning] ABT_xstream_get_cpubind failed to get cpubind (ret = 51)
[2020-12-01 18:35:22.808] [warning] ABT_xstream_get_affinity failed to get affinity (ret = 51)
[2020-12-01 18:35:22.808] [warning] ABT_xstre

## Accessing the service using a client

In [15]:
import bedrock.client

In [16]:
address = None # replace this
client = bedrock.client.Client('na+sm')
service_handle = client.create_service_handle(address)
service_handle.get_config()

'{"abt_io":[{"name":"my_abtio","pool":"my_rpc_pool"}],"bedrock":{"pool":"__primary__","provider_id":0},"libraries":{"module_a":"/projects/mochi/bedrock/build/examples/libexample-module-a.so","module_b":"/projects/mochi/bedrock/build/examples/libexample-module-b.so"},"margo":{"argobots":{"abt_mem_max_num_stacks":8,"abt_thread_stacksize":2097152,"pools":[{"access":"mpmc","kind":"fifo_wait","name":"__primary__"},{"access":"mpmc","kind":"fifo","name":"my_rpc_pool"}],"version":"1.0","xstreams":[{"affinity":[],"cpubind":-1,"name":"__primary__","scheduler":{"pools":[0],"type":"basic_wait"}},{"affinity":[],"cpubind":-1,"name":"my_xstream_0","scheduler":{"pools":[1],"type":"basic"}},{"affinity":[],"cpubind":-1,"name":"my_xstream_1","scheduler":{"pools":[1],"type":"basic"}},{"affinity":[],"cpubind":-1,"name":"my_xstream_2","scheduler":{"pools":[1],"type":"basic"}},{"affinity":[],"cpubind":-1,"name":"my_xstream_3","scheduler":{"pools":[1],"type":"basic"}}]},"enable_diagnostics":false,"enable_prof